In [1]:
import os

In [2]:
import pandas as pd

In [7]:
%pwd

'd:\\ML\\Data SC\\ML\\MLOPS try project\\salary-kaggle-project\\notebook'

In [8]:
os.chdir("../")

In [9]:
%pwd

'd:\\ML\\Data SC\\ML\\MLOPS try project\\salary-kaggle-project'

In [10]:
data = pd.read_csv("artifacts/data_ingestion/salaries.csv")

In [12]:
data.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2025,SE,FT,Solutions Engineer,214000,USD,214000,US,100,US,M
1,2025,SE,FT,Solutions Engineer,136000,USD,136000,US,100,US,M
2,2025,MI,FT,Data Engineer,158800,USD,158800,AU,0,AU,M
3,2025,MI,FT,Data Engineer,139200,USD,139200,AU,0,AU,M
4,2025,EN,FT,Data Engineer,90000,USD,90000,US,0,US,M


In [16]:
data.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64